<a href="https://colab.research.google.com/github/dubesar/coloi-1/blob/master/Betaversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [31]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [17]:
!ls

drive  sample_data


In [18]:
cd drive

/content/drive


In [21]:
cd ..

/content


In [22]:
cd ..

/


In [0]:
X = []
for filename in os.listdir('content/drive/My Drive/ds-medium'):
    X.append(img_to_array(load_img('content/drive/My Drive/ds-medium/'+filename)))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [0]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [40]:
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model      
tensorboard = TensorBoard(log_dir="/output/beta_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=10)

Epoch 1/10
10/10 [==============================] - 117s 12s/step - loss: 0.0096
Epoch 2/10
10/10 [==============================] - 113s 11s/step - loss: 0.0103
Epoch 3/10
10/10 [==============================] - 113s 11s/step - loss: 0.0110
Epoch 4/10
10/10 [==============================] - 112s 11s/step - loss: 0.0088
Epoch 5/10
10/10 [==============================] - 113s 11s/step - loss: 0.0093
Epoch 6/10
10/10 [==============================] - 112s 11s/step - loss: 0.0123
Epoch 7/10
10/10 [==============================] - 112s 11s/step - loss: 0.0094
Epoch 8/10
10/10 [==============================] - 112s 11s/step - loss: 0.0101
Epoch 9/10
10/10 [==============================] - 113s 11s/step - loss: 0.0082
Epoch 10/10
10/10 [==============================] - 112s 11s/step - loss: 0.0112


In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [43]:
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

10/10 [==============================] - 3s 250ms/step
0.010118258185684681


In [44]:
color_me = []
for filename in os.listdir('content/drive/My Drive/Validate'):
	color_me.append(img_to_array(load_img('content/drive/My Drive/Validate/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
	cur = np.zeros((256, 256, 3))
	cur[:,:,0] = color_me[i][:,:,0]
	cur[:,:,1:] = output[i]
	imsave("content/img_"+str(i)+".png", lab2rgb(cur))

/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
